<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Обоснование-метода-шифрования" data-toc-modified-id="Обоснование-метода-шифрования-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обоснование метода шифрования</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#GridSearch" data-toc-modified-id="GridSearch-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>GridSearch</a></span></li></ul></div>

# Защита персональных данных клиентов

<font color='purple'> 
Необходимо защитить данные клиентов страховой компании. Нужен такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Тем не менее при преобразовании качество моделей машинного обучения не должно ухудшиться, что необходимо доказать. Подбирать наилучшую модель не требуется.

В ходе работы обнаружено, что, если независимые переменные преобразовать в матрицу и умножить на обратимую матрицу-ключ, качество предсказания останется прежним.

В работе использованы библиотеки pandas, numpy, sklearn. В качестве модели машинного обучения выбрана линейная регрессия, ключевая метрика - R2. Приведено математическое доказательство неизменности предсказаний зависимой переменной при использовании зашифрованных и оригинальных данных. С помощью GridSearch незначительно улучшена ключевая метрика.

## Загрузка данных

Импортируем модули, которые нам понадобятся для дальнейшей работы.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

Читаем базу данных.

In [ ]:
df = pd.read_csv('/datasets/insurance.csv', sep= ',')
display(df.head().T)
df.info()

,0,1,2,3,4
Пол,1.0,0.0,0.0,0.0,1.0
Возраст,41.0,46.0,29.0,21.0,28.0
Зарплата,49600.0,38000.0,21000.0,41700.0,26100.0
Члены семьи,1.0,1.0,0.0,2.0,0.0
Страховые выплаты,0.0,1.0,0.0,0.0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Мы получили базу данных из 5 признаков. Всего в базе 5000 записей. Данные предобработаны. Названия столбцов не соответствуют стандарту PEP8, однако это не помешает дальнейшей работе.

## Обоснование метода шифрования

<font color='purple'> 
Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

<font color='purple'> 
Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

  $$ w = (X^T X)^{-1} X^T y $$ 

<font color='purple'> 
Признаки умножают на обратимую матрицу. Изменится ли качество линейной регрессии?

**Ответ:** Не изменится.

**Обоснование:** 

Умножим матрицу признаков $Х$ на обратимую матрицу $Р$. Тогда формула обучения будет такой:

$$
w1 = ((XP)^T(XP))^{-1} (XP)^Ty
$$

Раскроем скобки:

$$
w1 = (P^T*X^T*X*P)^{-1}*P^T*X^T*y
$$

$$
w1 = P^{-1}*(X^T*X)^{-1}*(P^T)^{-1}*P^T*X^T*y
$$

Матрицы $(P^T)^{-1}$ и $P^T$ при умножении сокращаются:

$$
w1 = P^{-1}*(X^T*X)^{-1}*X^T*y
$$

Правая часть этого уравнения $(X^TX)^{-1}X^Ty$ - это и есть исходное $w$.

$$
w1 = P^{-1}*w
$$

Тогда новые предсказания $a1$ для матрицы признаков $X$, умноженной на $Р$, будут рассчитываться по формуле

$$
a1 = X*P  *  w1
$$

$$
a1 = X * P * P^{-1} * w
$$

$$
a1 = X * w = a
$$

Таким образом, предсказания будут идентичны, если переменные-предикторы будут как расшифрованы, так и зашифрованы с помощью умножения матрицы признаков на случайную обратимую матрицу.

## Алгоритм преобразования

**Алгоритм**

Мы доказали, что предсказываемый признак не изменится, если матрицу предикторов умножить на случайную обратимую матрицу. Поэтому предлагаемый алгоритм следующий:
1. Исследуемый признак отделяется и определяется как переменная y
2. Оставшиеся 4 признака определяются как список списков X и преобразуется в матрицу
3. Генерируется случайная квадратная обратимая матрица Р
4. Матрица Х умножается на Р
5. X и y разделяются на тренировочный и тестовый набор данных

Далее весь набор данных передаётся для построения моделей.

**Обоснование**

Ранее мы доказали валидность шифрования исходных данных с помощью случайной обратимой матрицы. Предсказания целевого признака будут идентичны, используется умножение на матрицу-ключ или нет. Поэтому отделить целевой признак можно до шифрования. Прочие шаги - разделение признаков на целевой и предикторы, разделение на тренировочный и тестовый набор данных - необходимые этапы подготовки данных для построения модели. Четвёртый этап (умножение матрицы Х на Р), собственно, является этапом шифрования данных.

## Проверка алгоритма

Чтобы проверить работу описанного алгоритма, построим сначала модель линейной регрессии на чистых, незашифрованных данных. Мерой качества регрессии является метрика R2. Она понадобится нам в дальнейшем для сравнения двух моделей.

In [ ]:
X = df.drop(['Страховые выплаты'], axis=1)
y = df['Страховые выплаты']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)
print(X.shape, y.shape)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5000, 4) (5000,)
(4000, 4) (4000,)
(1000, 4) (1000,)


In [ ]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('Метрика R2 (доля объяснённых случаев) для модели линейной регрессии, построенной на открытых данных:',
      r2_score(y_test, predictions))

Метрика R2 (доля объяснённых случаев) для модели линейной регрессии, построенной на открытых данных: 0.4376399647442131


Полученная линия регрессии объясняет немного введённых в базу случаев, но сделать лучшую модель не является целью данного исследования. Мы вернёмся к полученному значению позже.


Преобразуем предикторы Х в матрицу, что необходимо для выбранного нами алгоритма шифрования:

In [ ]:
X_train = X_train.values
X_test = X_test.values
X_train

array([[0.00e+00, 3.00e+01, 3.47e+04, 1.00e+00],
       [0.00e+00, 1.90e+01, 5.44e+04, 1.00e+00],
       [1.00e+00, 3.60e+01, 3.64e+04, 2.00e+00],
       ...,
       [0.00e+00, 2.70e+01, 4.44e+04, 2.00e+00],
       [1.00e+00, 5.10e+01, 2.97e+04, 0.00e+00],
       [0.00e+00, 3.50e+01, 3.74e+04, 0.00e+00]])

Сгенерируем обратимую матрицу-ключ шифрования. Нам нужна матрица 4х4, т.к. у нас 4 признака-предиктора, а матрица должна быть квадратной.

In [ ]:
A = np.random.normal(size = (4,4))
A

array([[ 6.82212550e-01,  3.92058716e-01, -5.54339078e-01,
        -1.31235635e+00],
       [-1.63172427e+00,  1.22488633e+00, -2.03049354e+00,
         3.71774744e-01],
       [ 5.87546902e-01,  4.91070046e-01,  1.98314779e-03,
        -7.45670855e-01],
       [-3.79310359e-01,  3.17114376e-01, -1.13262825e+00,
         5.61702846e-01]])

Если случайно сгенерированная матрица А не является обратимой, следующий код выдаст ошибку:

In [ ]:
A_1 = np.linalg.inv(A)
A_1

array([[-0.18684058, -0.63927101,  0.9435296 ,  1.23913547],
       [-1.144762  ,  0.30193669,  2.18246417,  0.02280887],
       [-0.70575995,  0.14762748,  0.71241769, -0.80089252],
       [-0.90299299, -0.30447331,  0.84155563,  0.98926043]])

In [ ]:
def generate_invertible_matrix(size):
    try:
        matrix = np.random.normal(size=(size, size))
# проверим матрицу на обратимость, если нет, пробуем сгенерировать еще раз
# таким образом гарантируем, что матрица стопроцентно будет обратимой
        np.linalg.inv(matrix)
    except np.linalg.LinAlgError:
        matrix = generate_invertible_matrix()
    
    return matrix

Умножим матрицу с признаками-предикторами на матрицу-ключ А, то есть зашифруем их.

In [ ]:
X_train_encrypted = X_train @ A
X_test_encrypted = X_test @ A
X_train_encrypted

array([[ 2.03385465e+04,  1.70771943e+04,  6.76779389e+00,
        -2.58630637e+04],
       [ 3.19311694e+04,  2.67378004e+04,  6.81712342e+01,
        -4.05568691e+04],
       [ 2.13278888e+04,  1.79200719e+04, -3.73078343e+00,
        -2.71292242e+04],
       ...,
       [ 2.60422673e+04,  2.18372162e+04,  3.09631798e+01,
        -3.30966246e+04],
       [ 1.73676073e+04,  1.46476416e+04, -4.52100202e+01,
        -2.21287762e+04],
       [ 2.19171438e+04,  1.84088907e+04,  3.10245348e+00,
        -2.78750779e+04]])

Значения действительно стали сильно отличаться пола, возраста заплаты и количества членов семьи в исходнике. Обучим модель:

In [ ]:
model_encrypted = LinearRegression()
model_encrypted.fit(X_train_encrypted, y_train)
predictions_encrypted = model_encrypted.predict(X_test_encrypted)
print('Метрика R2 (доля объяснённых случаев) для модели линейной регрессии, построенной на шифрованных данных:',
      r2_score(y_test, predictions_encrypted))

Метрика R2 (доля объяснённых случаев) для модели линейной регрессии, построенной на шифрованных данных: 0.43763996474424893


Метрики R2 зашифрованной и исходной модели раличаются начиная с 14 знака после запятой, будем считать их равными. Различия в предсказаниях тоже можно считать тождественными, т.к. разница начинается с 13 знака после запятой, что соответствует особенносям хранения данных и округления в Python.

In [ ]:
predictions_encrypted[:10] - predictions[:10]

array([-2.40585329e-12, -2.37321274e-12, -2.25963692e-12, -2.26041408e-12,
       -2.51776378e-12, -2.25319763e-12, -2.43449705e-12, -2.30260255e-12,
       -2.31104025e-12, -2.26263452e-12])

## Вывод
Мы доказали тождественность предсказаний на зашифрованных и исходных данных у модели линейной регрессии. Определён алгоритм шифрования с помощью преобразования предикторов в матрицу и умножения данной матрицы на случайно сгенерированную обратимую квадратную матрицу той же ширины. Качество линейной регрессии до и после преобразования является идентичным по метрике R2  и расно 0,44.

## GridSearch

Попробуем сменить модель для получения более высокого R2. В качестве модели возьмём случайный лес, параметры подберём с помощью GridSearch.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

param_grid = {
    'bootstrap': [False, True],
    'max_depth': [80, 90, 100, 110, 120],
    'max_features': [2, 3, 4],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

model = RandomForestRegressor()

grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 5, n_jobs = -1, verbose = 2)

grid_search.fit(X_train, y_train)
grid_search.best_params_

Fitting 5 folds for each of 1080 candidates, totalling 5400 fits
[CV] bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.1s
[CV] bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV]  bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.1s
[CV] bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.1s
[CV] bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.1s
[CV] bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100 
[CV]  bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=100, total=   0.1s
[CV] bootstrap=False, max_depth=80, max_features=2, min_samples_leaf=3, min_samples_split=8, n_estimators=200 
[CV]  bootstrap=False, max_depth=80, max_features=2,

[Parallel(n_jobs=-1)]: Done 5400 out of 5400 | elapsed: 49.4min finished


{'bootstrap': False,
 'max_depth': 80,
 'max_features': 4,
 'min_samples_leaf': 3,
 'min_samples_split': 8,
 'n_estimators': 100}

Подставим лучшие параметры в модель, обучим её заново и посчитаем ключевую метрику.

In [ ]:
model_forest = RandomForestRegressor(
    random_state = 42,
    bootstrap = False,
    max_depth = 80,
    max_features = 4,
    min_samples_leaf = 3,
    min_samples_split = 8,
    n_estimators = 100)
model_forest.fit(X_train, y_train)
predictions_forest = model_forest.predict(X_test)
print('Метрика R2 (доля объяснённых случаев) для случайного леса с наилучшими параметрами:',
      r2_score(y_test, predictions_forest))

Метрика R2 (доля объяснённых случаев) для случайного леса с наилучшими параметрами: 0.9996646995708155


Результат получился высоким, однако сравним его с обычной моделью случайного леса без подбора гиперпараметров:

In [ ]:
model_forest = RandomForestRegressor(random_state = 42)
model_forest.fit(X_train, y_train)
predictions_forest = model_forest.predict(X_test)
print('Метрика R2 (доля объяснённых случаев) для случайного леса без подбора параметров:',
      r2_score(y_test, predictions_forest))

Метрика R2 (доля объяснённых случаев) для случайного леса без подбора параметров: 0.9996781115879828


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Уровень R2 оказался таким же.